In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import torch
import torch.utils.data as Data
from torch import nn
from torch.nn import functional as F
# import matplotlib.pyplot as plt
import time
import sys
import scipy
from utilities import *
from models import *
from timeit import default_timer
import numpy as np

from Adam import Adam

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
SEED_SET(1)

In [18]:
operator = 'nqf' # f, qf, nqf
# if operator == 'f':
#     neumann_test = True

model_type = 'UNO' # FNO, UNO

DC_in = True
FNO_act = 'gelu'
CNN_act = 'gelu'

times = 1

data_weight = 1
PINN_weight = 0.05

num_layer = 3

data_name = 'qT_01_fGRF_23_k20'

data_train_name = './data1/ftrain_' + data_name + '.npz'
data_test_name = './data1/ftest_' + data_name + '.npz'

f_real = True

In [19]:
save_model = True
if save_model:
    filename_model = model_type + '_' + operator + '_' + data_name + '_256_dw1_pw005_git1.pth'
    filename_data = model_type + '_' + operator + '_' + data_name + '_256_dw1_pw005_git1.npz'

In [20]:
# load data
if operator == 'qf':
    if f_real:
        cin = 2
    else:
        cin = 3
else:
    if f_real:
        cin = 1
    else:
        cin = 2

# k, N_sample, N, x_train, y_train, ini = load_data(data_train_name, times, operator, f_real, device)
# k, N_test, N, x_test, y_test, ini_test = load_data(data_test_name, times, operator, f_real, device)

k, N_sample, N, x_train, y_train = load_data(data_train_name, times, operator, f_real, device)
k, N_test, N, x_test, y_test = load_data(data_test_name, times, operator, f_real, device)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

torch.Size([1000, 2, 257, 257])
torch.Size([1000, 2, 257, 257])
torch.Size([100, 2, 257, 257])
torch.Size([100, 2, 257, 257])


In [21]:
Batch_size = 20

train_data = Data.TensorDataset(x_train, y_train)
N_Batch = N_sample/Batch_size
train_loader = Data.DataLoader(dataset = train_data, batch_size = Batch_size, shuffle = False)

test_data = Data.TensorDataset(x_test, y_test)
N_Batch = N_test/Batch_size
test_loader = Data.DataLoader(dataset = test_data, batch_size = Batch_size, shuffle = False)

In [22]:
modes = 12
width = 32

if model_type == 'FNO':
    if operator == 'nqf':
        model = Neumann_FNO(N, k, modes1 = modes, modes2 = modes, width = width, FNO_act = FNO_act, CNN_act = CNN_act, DC_in = DC_in).to(device)
    else:
        model = FNO2d(modes1 = modes, modes2 = modes, width = width, FNO_act = FNO_act, CNN_act = CNN_act, DC_in = DC_in, cin = cin).to(device)
else:
    if operator == 'nqf':
        model = Neumann_MyUNO(N, k, modes1 = modes, modes2 = modes, width = width, FNO_act = FNO_act, CNN_act = CNN_act, num_layer = num_layer, DC_in = DC_in).to(device)
    else:
        model = MyUNO(N, k, modes1 = modes, modes2 = modes, width = width, FNO_act = FNO_act, CNN_act = CNN_act, DC_in = DC_in, num_layer = num_layer, cin = cin).to(device)

print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in model.parameters())))

Total number of paramerters in networks is 3558813  


In [23]:
epochs = 500
step_size = 100

gamma = 0.5
lr = 0.001

optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# start_lr = 2e-3
# end_lr = 1e-5
# optimizer = torch.optim.Adam(model.parameters(), lr=start_lr, betas=(0.9, 0.999), eps=1e-8)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=end_lr)

In [24]:
train_er, test_er, PINN_train_er, PINN_test_er = train_model_PINN(train_loader, test_loader, model, epochs, optimizer, scheduler, N, k,
                                                                  operator = operator, data_weight = data_weight, PINN_weight = PINN_weight)

Epoch [1/100], train: 1.0307, PINN_train: 11.0486, test: 0.9728, PINN_test: 1.62, Time per epoch: 8.8207
Epoch [10/100], train: 0.4875, PINN_train: 1.6972, test: 0.4793, PINN_test: 1.652, Time per epoch: 7.3774
Epoch [20/100], train: 0.1993, PINN_train: 1.2977, test: 0.1952, PINN_test: 1.382, Time per epoch: 6.5545
Epoch [30/100], train: 0.1174, PINN_train: 0.9884, test: 0.1193, PINN_test: 0.995, Time per epoch: 6.4912
Epoch [40/100], train: 0.0666, PINN_train: 0.8434, test: 0.07289, PINN_test: 0.8574, Time per epoch: 6.4060
Epoch [50/100], train: 0.0536, PINN_train: 0.7018, test: 0.05614, PINN_test: 0.7252, Time per epoch: 7.0325
Epoch [60/100], train: 0.0413, PINN_train: 0.5936, test: 0.04694, PINN_test: 0.6126, Time per epoch: 7.5037
Epoch [70/100], train: 0.0401, PINN_train: 0.5722, test: 0.04716, PINN_test: 0.5731, Time per epoch: 6.5641
Epoch [80/100], train: 0.0336, PINN_train: 0.5266, test: 0.03864, PINN_test: 0.557, Time per epoch: 6.4670
Epoch [90/100], train: 0.0328, PINN_tr

In [25]:
ave_train_error, ave_test_error = model_test(train_loader, test_loader, model)

print('ave_train_error: ', ave_train_error)
print('ave_test_error: ', ave_test_error)

ave_train_error:  0.03022108054161072
ave_test_error:  0.0366783618927002


In [26]:
if save_model:
    model = model.cpu()
    torch.save(model.state_dict(), filename_model)

    np.savez(filename_data, width=width, modes=modes, times=times, operator=operator, model_type=model_type,
             FNO_act=FNO_act, CNN_act=CNN_act, cin=cin, f_real=f_real, DC_in=DC_in,
             Batch_size=Batch_size, step_size=step_size, gamma=gamma, learning_rate=lr, epochs=epochs,
             data_weight=data_weight, PINN_weight=PINN_weight, num_layer=num_layer,
             train_er=train_er, test_er=test_er, PINN_train_er=PINN_train_er, PINN_test_er=PINN_test_er,
             ave_train_error=ave_train_error, ave_test_error=ave_test_error)

### 